In [1]:
import numpy as np
from PIL import Image
import matplotlib.pylab as plt
from scipy import stats

In [59]:
Calibracion = 81.9010

In [57]:
def variables(matrix,Cal):
    x=[]
    y=[]
    L_filas = len(matrix[:,0])
    L_columnas = len(matrix[0,:])
    for i in range(L_filas):
        for j in range(L_columnas):
            if matrix[i,j]==255:
                x.append(j)
                y.append(L_filas-1-i)
    x=np.asarray(x)/Cal
    y=np.asarray(y)/Cal
    grad, inter, r_value, p_value, std_err = stats.linregress(x,y)
    return x, y, grad

In [58]:
def contorno(matrix,Cal):
    XM=[]
    Xm=[]
    YM=[]
    Ym=[]
    L_filas = len(matrix[:,0])
    L_columnas = len(matrix[0,:])
    #Maximos
    for j in range(L_columnas):
        for i in range(L_filas):
            if matrix[i,j]==255:
                XM.append(j)
                YM.append(L_filas-1-i)
                break
    #Minimos            
    for j in range(L_columnas):
        for i in range(L_filas):
            if matrix[L_filas-1-i,j]==255:
                Xm.append(j)
                Ym.append(i)
                break

    XM=np.asarray(XM)/Cal
    YM=np.asarray(YM)/Cal
    Xm=np.asarray(Xm)/Cal
    Ym=np.asarray(Ym)/Cal
    return XM, YM, Xm, Ym

In [74]:
def diametro_y_altura(archivo,Cal):
    imagen = Image.open(archivo)
    matrix = np.array(imagen)
    x, y, grad = variables(matrix,1)
    imagen_rotada = imagen.rotate(-np.arctan(grad)*180/np.pi, expand=True)
    matrix_rotada = np.array(imagen_rotada)
    xM, yM, xm, ym = contorno(matrix_rotada,Cal)
    diametros = yM-ym
    D = np.mean(diametros)
    s = np.sqrt(np.var(diametros))
    D_filtrado = []
    for i in diametros:
        if i<D+2*s and i>D-2*s:
            D_filtrado.append(i)
    D_prom = np.mean(D_filtrado)
    err = np.sqrt(np.var(D_filtrado))
#     plt.plot(xM,diametros)
#     plt.plot(xM,np.ones_like(xM)*D,'m')
#     plt.plot(xM,np.ones_like(xM)*D_prom,'r')
#     plt.plot(xM,np.ones_like(xM)*(D-2*s),'g')
#     plt.plot(xM,np.ones_like(xM)*(D+2*s),'g')
#     plt.ylim([D-2*s,D+2*s])

#ALTURA
    matrix_transp = np.matrix.transpose(matrix_rotada)
    xMtr, yMtr, xmtr, ymtr = contorno(matrix_transp,Cal)
    longitudes = yMtr-ymtr
    L = np.mean(longitudes)
    sL = np.sqrt(np.var(longitudes))
    L_filtrado = []
    for i in longitudes:
        if i<L+2*sL and i>L-2*sL:
            L_filtrado.append(i)
    L_prom = np.mean(L_filtrado)
    errL = np.sqrt(np.var(L_filtrado))
    return D_prom, err, L_prom, errL

In [79]:
diametro_y_altura('CilindroBW50.png',Calibracion)

(2.1773343773797027,
 0.05526594289706265,
 27.234594700193412,
 0.6744276094789862)